<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/dicom.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>

<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/dicom.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td> 

# DICOM Annotation Import

* Annotations must be created and uploaded using NDJSON
* Supported annotations that can be uploaded through the SDK:
    * Polyline
    * Segmentation masks     
* **NOT** supported:
    * Bounding box
    * Point
    * Polygons
    * Free form text classifications
    * Radio classifications 
    * Checklist classifications

### Setup

In [ ]:
!pip install -q 'labelbox[data]'


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import labelbox as lb
import labelbox.types as lb_types
import uuid

### Replace with your API key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
# Add your api key
API_KEY=None
client = lb.Client(api_key=API_KEY)

## Supported annotations for DICOM


In [ ]:
######## Polyline ########
polyline_annotation = [
    lb_types.DICOMObjectAnnotation(
        name="line_dicom",
        group_key=lb_types.GroupKey.AXIAL,
        frame=1,
        value=lb_types.Line(points=[
            lb_types.Point(x=10, y=10),
            lb_types.Point(x=200, y=20),
            lb_types.Point(x=250, y=250)
        ]),
        segment_index=0,
        keyframe=True,
    ),
    lb_types.DICOMObjectAnnotation(
        name="line_dicom",
        group_key=lb_types.GroupKey.AXIAL,
        frame=20,
        value=lb_types.Line(points=[
            lb_types.Point(x=10, y=10),
            lb_types.Point(x=200, y=10),
            lb_types.Point(x=300, y=300)
        ]),
        segment_index=1,
        keyframe=True,
    ),    
]

polyline_annotation_ndjson = {
  'name': 'line_dicom',
  'groupKey': 'axial', # should be 'axial', 'sagittal', or 'coronal'
  'segments': [
    {
    'keyframes': [{
        'frame': 1,
        'line': [
            {'x': 10, 'y': 10},
            {'x': 200, 'y': 20},
            {'x': 250, 'y': 250},
        ]
    }]},
    {
    'keyframes' : [{
        'frame': 20,
        'line': [
            {'x': 10, 'y': 10},
            {'x': 200, 'y': 10},
            {'x': 300, 'y': 300},
        ]
    }]}
    ],
}

In [ ]:
######## Segmentation Masks ########

mask_annotation = [
    lb_types.DICOMMaskAnnotation(
    group_key='axial',
    frames=[
        lb_types.MaskFrame(
            index=1,
            instance_uri="https://storage.googleapis.com/labelbox-datasets/dicom-sample-data/sample-mask-1.png"
        ),
        lb_types.MaskFrame(
            index=5,
            instance_uri="https://storage.googleapis.com/labelbox-datasets/dicom-sample-data/sample-mask-1.png"
        )
    ],
    instances=[
        lb_types.MaskInstance(
            color_rgb=(255, 255, 255),
            name="segmentation_mask_dicom"
        )
    ])
]

mask_annotation_ndjson = {
    'groupKey': 'axial',
    'masks': {
        'frames': [{
            'index': 1,
            'instanceURI': "https://storage.googleapis.com/labelbox-datasets/dicom-sample-data/sample-mask-1.png"
        }, {
            'index': 5,
            'instanceURI': "https://storage.googleapis.com/labelbox-datasets/dicom-sample-data/sample-mask-1.png"
        }],
        'instances': [
            {
                'colorRGB': (255, 255, 255),
                'name': 'segmentation_mask_dicom'
            }
        ]
    },
}


## Upload Annotations - putting it all together

### Step 1: Import data rows into Catalog

In [ ]:
global_key = "sample-dicom-1.dcm"
asset = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/dicom-sample-data/sample-dicom-1.dcm", 
    "global_key": global_key,
}

dataset = client.create_dataset(name="dicom_demo_dataset")
task = dataset.create_data_rows([asset])
task.wait_till_done()
print("Errors :",task.errors)
print("Failed data rows:" ,task.failed_data_rows)

Errors : None
Failed data rows: None


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the line annotation above, we provided the `name` as `line_dicom`. Now, when we setup our ontology, we must ensure that the name of my line tool is also `line_dicom`. The same alignment must hold true for the other tools and classifications we create in our ontology.


[Documentation for reference ](https://docs.labelbox.com/reference/import-text-annotations)

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="segmentation_mask_dicom"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="line_dicom"),
    ]
)

ontology = client.create_ontology("Ontology DICOM Annotations", ontology_builder.asdict(), media_type=lb.MediaType.Dicom)

### Step 3: Create a labeling project 
Connect the ontology to the labeling project.

In [ ]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="dicom_project_demo", media_type=lb.MediaType.Dicom)

## connect ontology to your project
project.setup_editor(ontology)

Default createProject behavior will soon be adjusted to prefer batch projects. Pass in `queue_mode` parameter explicitly to opt-out for the time being.


### Step 4: Send a batch of data rows to the project

In [ ]:
# Create batches

# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-dicom-demo", # Each batch in a project must have a unique name
  global_keys=[global_key], # a list of data row objects, data row ids or global keys
  priority=5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-03-29 15:21:44+00:00",
    "name": "first-batch-dicom-demo",
    "size": 0,
    "uid": "6a13c820-ce45-11ed-841e-4f0ab6db95bd",
    "updated_at": "2023-03-29 15:21:44+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above.

Labelbox supports two formats for the annotations payload: NDJSON and Python Annotation types.

#### Python Annotation Types

In [ ]:

annotations_list = polyline_annotation + mask_annotation
labels = [
    lb_types.Label(
        data=lb_types.DicomData(global_key=global_key),
        annotations=annotations_list
    )
]

#### NDJSON annotations
Here we create the complete `label_ndjson` payload of annotations. There is one annotation for each *reference to an annotation* that we created above.

In [ ]:
label_ndjson = []

for annotation in [
    polyline_annotation_ndjson, 
    mask_annotation_ndjson
]:      
  annotation.update({
      'dataRow': {
          'globalKey': global_key
      }
  })
  label_ndjson.append(annotation)


### Step 6: Upload annotations to a project as pre-labels or completed labels
For the purpose of this tutorial only run one of the label imports at once, otherwise the previous import might get overwritten.

#### Model-Assisted Labeling (MAL)

In [ ]:
# Upload MAL label for this data row in project
upload_job_mal = lb.MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job-" + str(uuid.uuid4()), 
    predictions=labels)

upload_job_mal.wait_until_done();
print("Errors:", upload_job_mal.errors)
print("Status of uploads: ", upload_job_mal.statuses)
print("   ")

/Users/ibrahim/workspace/labelbox-python/labelbox/data/serialization/ndjson/label.py:177: UserWarning: Nested classifications are not currently supported
                    for video object annotations
                    and will not import alongside the object annotations.
  warnings.warn(


Errors: []
Status of uploads:  [{'uuid': 'd193960f-1c7b-4493-87d1-8a55b2405748', 'dataRow': {'id': 'clftu3umv0384075r5poqey9o', 'globalKey': 'sample-dicom-1.dcm'}, 'status': 'SUCCESS'}, {'uuid': '8d0c454e-3385-44fc-97db-0516db3e7dc1', 'dataRow': {'id': 'clftu3umv0384075r5poqey9o', 'globalKey': 'sample-dicom-1.dcm'}, 'status': 'SUCCESS'}]
   


#### Label Import

In [ ]:
upload_job_label_import = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid, 
    name = "label_import_job-" + str(uuid.uuid4()),
    labels=labels
)

upload_job_label_import.wait_until_done()
print("Errors:", upload_job_label_import.errors)
print("Status of uploads: ", upload_job_label_import.statuses)
print("   ")

Errors: []
Status of uploads:  [{'uuid': '1cc42044-4f36-45af-8b05-da6c8dd5bf37', 'dataRow': {'id': 'clftu3umv0384075r5poqey9o', 'globalKey': 'sample-dicom-1.dcm'}, 'status': 'SUCCESS'}, {'uuid': '005e5d45-b3c2-49f5-bf82-aef760de7826', 'dataRow': {'id': 'clftu3umv0384075r5poqey9o', 'globalKey': 'sample-dicom-1.dcm'}, 'status': 'SUCCESS'}]
   


### Optional deletions for cleanup

In [ ]:
# Delete Project
# project.delete()
# dataset.delete()
